# DuckDB Introduction

In [54]:
import duckdb
con = duckdb.connect("camp.duckdb")

In [55]:
con.execute("""
    CREATE TABLE IF NOT EXISTS bank AS
    SELECT * FROM read_csv('bank-marketing.csv')
""")
con.execute("SHOW ALL TABLES").fetchdf()

,database,schema,name,column_names,column_types,temporary
0,camp,main,bank,"[index, age, job, marital, education, default,...","[BIGINT, BIGINT, VARCHAR, VARCHAR, VARCHAR, VA...",False


In [56]:
con.execute("SELECT * FROM bank WHERE duration < 100 LIMIT 5").fetchdf()

,index,age,job,marital,education,default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,9,25,services,single,high.school,no,yes,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,False
1,10,41,blue-collar,married,unknown,unknown,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,False
2,20,30,unemployed,married,high.school,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,False
3,25,35,technician,married,university.degree,no,no,yes,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,False
4,26,59,technician,married,unknown,no,yes,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,False


In [57]:
bank_duck = duckdb.read_csv("bank-marketing.csv",sep=",")


In [58]:
bank_duck

┌───────┬───────┬─────────────┬──────────┬─────────────────────┬─────────┬─────────┬─────────┬───────────┬─────────┬─────────────┬──────────┬──────────┬───────┬──────────┬─────────────┬──────────────┬────────────────┬───────────────┬───────────┬─────────────┬─────────┐
│ index │  age  │     job     │ marital  │      education      │ default │ housing │  loan   │  contact  │  month  │ day_of_week │ duration │ campaign │ pdays │ previous │  poutcome   │ emp.var.rate │ cons.price.idx │ cons.conf.idx │ euribor3m │ nr.employed │    y    │
│ int64 │ int64 │   varchar   │ varchar  │       varchar       │ varchar │ varchar │ varchar │  varchar  │ varchar │   varchar   │  int64   │  int64   │ int64 │  int64   │   varchar   │    double    │     double     │    double     │  double   │    int64    │ boolean │
├───────┼───────┼─────────────┼──────────┼─────────────────────┼─────────┼─────────┼─────────┼───────────┼─────────┼─────────────┼──────────┼──────────┼───────┼──────────┼─────────────┼─────

In [59]:
bank_duck = duckdb.read_csv("bank-marketing.csv",sep=",")
bank_duck.filter("duration < 100").limit(3).df()

,index,age,job,marital,education,default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,9,25,services,single,high.school,no,yes,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,False
1,10,41,blue-collar,married,unknown,unknown,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,False
2,20,30,unemployed,married,high.school,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,False


In [60]:
rel = con.table("bank")
rel.columns

['index',
 'age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed',
 'y']

In [61]:
rel.filter("duration < 100").project("job,education,loan").order("job").limit(3).df()

,job,education,loan
0,blue-collar,unknown,no
1,blue-collar,unknown,no
2,blue-collar,basic.9y,no


## Query Function

In [62]:
res = duckdb.query("""SELECT
                            job,
                            COUNT(*) AS total_clients_contacted,
                            AVG(duration) AS avg_campaign_duration,
                        FROM
                            'bank-marketing.csv'
                        WHERE
                            age > 30
                        GROUP BY
                            job
                        ORDER BY
                            total_clients_contacted DESC;""")
res.df()

,job,total_clients_contacted,avg_campaign_duration
0,admin.,26,245.807692
1,blue-collar,23,320.695652
2,technician,13,385.153846
3,management,9,283.444444
4,services,8,323.750000
5,unknown,4,239.250000
6,housemaid,3,273.333333
7,entrepreneur,3,601.666667
8,retired,2,258.000000
9,unemployed,2,345.000000


In [63]:
con.close() # closing the connection